In [9]:
import pandas as pd
import json
import engin
import models
import random
import warnings, traceback
warnings.filterwarnings("ignore")

In [2]:
with open('/Users/daniil/Documents/Работа/Платформа анализа данных/Прогноз/Auto_forecast/test_data_10000.json', "r", encoding="utf8") as f:
    data = json.load(f)

In [3]:
passed, failed = engin.Filter_for_dataset(data)
print('passed -', len(passed))
print('failed -', len(failed))

passed - 9679
failed - 321


In [4]:
df = pd.DataFrame(passed)

Разделим данные по частотностям:

In [11]:
# Инициализируем три пустых списка
ye_list = []
q_list = []
me_list = []

# Проходим по исходному списку
for item in passed:
    freq = item.get("pandas_frequency")
    if freq == "YE":
        ye_list.append(item)
    elif freq == "Q":
        q_list.append(item)
    elif freq == "ME":
        me_list.append(item)

print('Годовые данные: ',len(ye_list))
print('Квартальные данные: ',len(q_list))
print('Месячные данные: ',len(me_list))

Годовые данные:  4741
Квартальные данные:  120
Месячные данные:  4816


Так как функция автопрогноза требудет больших вычислительных мощностей, ее тестирование будет для случайно выбранных рядов из каждой категории

In [12]:
# Фиксируем seed для воспроизводимости
random.seed(42+3)

sample_ye = random.sample(ye_list, 30)
sample_q  = random.sample(q_list, 30)
sample_me = random.sample(me_list, 30)

In [13]:
# Функция для тестирования
def testing(data: list,
            Deep_research: bool):
    
    successful_results, raise_dic, warns = {}, {}, []
    
    for i in range(len(data)):
        df = pd.DataFrame(data)
        slice_id = df.loc[i, 'slice_id']
        Data = pd.DataFrame(df.loc[i, 'observations'])

        with warnings.catch_warnings(record=True) as wlist:
                warnings.simplefilter("always")                  # перехватываем все предупреждения
                try:
                    out = engin.Auto_forecast(data[i], Deep_research)
                    successful_results[slice_id] = out
                except Exception as e:
                    # Сохраняем возникшие исключения
                    raise_dic[slice_id] = {
                    "Error_test": e, 
                    "type": type(e).__name__,
                    "msg": str(e),
                    "traceback": traceback.format_exc()
                }
            # Сохраняем предупреждения из этого прогона
                for w in wlist:
                    warns.append({
                        "slice_id": slice_id,
                        "category": w.category.__name__,
                        "message": str(w.message),
                        "filename": w.filename,
                        "lineno": w.lineno,
                    })

        result = {
            'successful_results' : successful_results,
            'raise_dic' : raise_dic,
            'warns' : warns
        }
    return result


In [14]:
res_test_YE_F = testing(sample_ye, False)
res_test_YE_T = testing(sample_ye, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_YE_F['successful_results']))
print('Исключения c F: ', len(res_test_YE_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_YE_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_YE_T['successful_results']))
print('Исключения c T: ', len(res_test_YE_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_YE_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  30
Исключения c F:  0
Предупреждения c F:  2599
--------------------------------------------------------
Успешно c T:  30
Исключения c T:  0
Предупреждения c T:  6326


In [15]:
pd.DataFrame(res_test_YE_F['raise_dic'])#.iloc[3,0]

""


In [16]:
res_test_Q_F = testing(sample_q, False)
res_test_Q_T = testing(sample_q, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_Q_F['successful_results']))
print('Исключения c F: ', len(res_test_Q_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_Q_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_Q_T['successful_results']))
print('Исключения c T: ', len(res_test_Q_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_Q_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  30
Исключения c F:  0
Предупреждения c F:  7482
--------------------------------------------------------
Успешно c T:  30
Исключения c T:  0
Предупреждения c T:  17744


In [17]:
pd.DataFrame(res_test_Q_F['raise_dic'])#.iloc[3,2]

""


In [18]:
res_test_ME_F = testing(sample_me, False)
res_test_ME_T = testing(sample_me, True)
print('Статистика: ')
print('--------------------------------------------------------')
print('Успешно c F: ', len(res_test_ME_F['successful_results']))
print('Исключения c F: ', len(res_test_ME_F['raise_dic']))
print('Предупреждения c F: ', len(res_test_ME_F['warns']))
print('--------------------------------------------------------')
print('Успешно c T: ', len(res_test_ME_T['successful_results']))
print('Исключения c T: ', len(res_test_ME_T['raise_dic']))
print('Предупреждения c T: ', len(res_test_ME_T['warns']))

Статистика: 
--------------------------------------------------------
Успешно c F:  30
Исключения c F:  0
Предупреждения c F:  11647
--------------------------------------------------------
Успешно c T:  30
Исключения c T:  0
Предупреждения c T:  38041


In [19]:
pd.DataFrame(res_test_ME_T['raise_dic'])

""


Отключения моделей которые "падают"